In [1]:
import os
from dotenv import load_dotenv
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from operator import itemgetter

# 환경 변수 로드
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# 단계 1: 문서 로드(Load Documents)

# PDF 문서 로드
pdf_loader = PyPDFLoader("./data/체류민원.pdf")
pdf_docs = pdf_loader.load()

# URL 문서 로드
url_file_path = "./data/urls.txt"
with open(url_file_path, "r") as file:
    urls = file.read().splitlines()

url_loaders = [WebBaseLoader(url) for url in urls]
url_docs = []
for loader in url_loaders:
    url_docs.extend(loader.load())

# PDF 문서와 URL 문서를 합침
docs = pdf_docs + url_docs

# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)

# 단계 3: 임베딩(Embedding) 생성
embeddings = OpenAIEmbeddings()

# 단계 4: DB 생성(Create DB) 및 저장
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)
vector_store_file = "./data/faiss_index"
vectorstore.save_local(vector_store_file)

# 단계 5: 검색기(Retriever) 생성
retriever = vectorstore.as_retriever()

# 단계 6: 프롬프트 생성(Create Prompt)
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
    Use the following pieces of retrieved context to answer the question. 
    If you don't know the answer, just say that you don't know. 
    Answer in Korean.

#Previous Chat History:
{chat_history}

#Question: 
{question} 

#Context: 
{context} 

#Answer:"""
)

# 단계 7: 언어모델(LLM) 생성
llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125", temperature=0)

# 단계 8: 체인(Chain) 생성
chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "chat_history": itemgetter("chat_history"),
    }
    | prompt
    | llm
    | StrOutputParser()
)


# 세션 기록을 저장할 딕셔너리
store = {}

# 세션 ID를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_id):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# 대화를 기록하는 RAG 체인 생성
rag_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,  # 세션 기록을 가져오는 함수
    input_messages_key="question",  # 사용자의 질문이 템플릿 변수에 들어갈 key
    history_messages_key="chat_history",  # 기록 메시지의 키
)

In [2]:
# 첫 번째 질문 실행
first_question = rag_with_history.invoke(
    {"question": "유학생이 체류 기간 동안 아르바이트를 하기 위해 어떤 절차를 따라야 해?"},
    config={"configurable": {"session_id": "session_1"}},
)

print(first_question)

유학생이 아르바이트를 하기 위해서는 체류자격 외 활동허가를 받아야 합니다. 학부과정은 주당 20시간, 석박사과정 및 논문준비 중인 경우 주당 30시간까지 가능하며, 공휴일 및 방학 중에는 무제한으로 허용됩니다.


In [8]:
# 이어진 질문 실행
follow_up_question = rag_with_history.invoke(
    {"question": "나는 지금 대학교 2학년인데, 충족해야할 기준 한국어 성적이 어느정도 돼?"},
    config={"configurable": {"session_id": "session_1"}},
)
print(follow_up_question)

학사 1~2학년의 경우에는 TOPIK 3급, 사회통합프로그램 3단계 이상 이수 또는 사전평가 61점 이상, 세종학당 중급1 이상 이수 수준이 필요합니다.


In [9]:
# 이어진 질문 실행
follow_up_question = rag_with_history.invoke(
    {"question": "나는 지금 대학교 3학년으로 올라가면, 충족해야할 기준 한국어 성적이 바뀌어?"},
    config={"configurable": {"session_id": "session_1"}},
)
print(follow_up_question)

3학년으로 올라가면, 한국어 성적 기준은 TOPIK 4급, 사회통합프로그램 4단계 이상 이수 또는 사전평가 81점 이상, 세종학당 중급2 이상 이수 수준이 필요합니다.


In [10]:
# 이어진 질문 실행
follow_up_question = rag_with_history.invoke(
    {"question": "더 자세히 말해. 관련된 정보가 쓰여있는 웹페이지가 있어?"},
    config={"configurable": {"session_id": "session_1"}},
)
print(follow_up_question)

해당 정보는 2024년 6월 15일 기준으로 작성된 것이며, 법적 효력을 갖는 유권해석의 근거가 되지 않습니다. 더 자세한 정보를 원하시면 해당 링크를 참고하시기 바랍니다. (https://www.easylaw.go.kr/CSP/CnpClsMain.laf?popMenu=ov&csmSeq=508&ccfNo=3&cciNo=7&cnpClsNo=1&menuType=cnpcls&search_put=%EC%9C%A0%ED%95%99%EC%83%9D)


In [3]:
# 이어진 질문 실행
follow_up_question = rag_with_history.invoke(
    {"question": "일정 수준의 한국어 능력이 어느 정도야?"},
    config={"configurable": {"session_id": "session_1"}},
)
print(follow_up_question)

일정 수준의 한국어 능력은 TOPIK 2급, 사회통합프로그램 2단계 이상 이수 또는 사전평가 41점 이상, 세종학당 중급1 이상 이수 수준입니다.


In [4]:
# 이어진 질문 실행
follow_up_question = rag_with_history.invoke(
    {"question": "더 자세히 말해. 관련된 정보가 쓰여있는 웹페이지가 있어?"},
    config={"configurable": {"session_id": "session_1"}},
)
print(follow_up_question)

해당 정보는 2024년 6월 15일 기준으로 작성된 것이며, 법적 효력을 갖는 유권해석의 근거가 되지 않습니다. 더 자세한 정보를 원하시면 해당 링크를 참고하시기 바랍니다. (https://www.easylaw.go.kr/CSP/CnpClsMain.laf?popMenu=ov&csmSeq=508&ccfNo=3&cciNo=7&cnpClsNo=1&menuType=cnpcls&search_put=%EC%9C%A0%ED%95%99%EC%83%9D)


In [11]:
# 대화 내역 확인
session_history = get_session_history("session_1")
print("Chat History:")
for message in session_history.messages:
    print(message)

Chat History:
content='유학생이 체류 기간 동안 아르바이트를 하기 위해 어떤 절차를 따라야 해?'
content='유학생이 아르바이트를 하기 위해서는 체류자격 외 활동허가를 받아야 합니다. 학부과정은 주당 20시간, 석박사과정 및 논문준비 중인 경우 주당 30시간까지 가능하며, 공휴일 및 방학 중에는 무제한으로 허용됩니다.'
content='일정 수준의 한국어 능력이 어느 정도야?'
content='일정 수준의 한국어 능력은 TOPIK 2급, 사회통합프로그램 2단계 이상 이수 또는 사전평가 41점 이상, 세종학당 중급1 이상 이수 수준입니다.'
content='더 자세히 말해. 관련된 정보가 쓰여있는 웹페이지가 있어?'
content='해당 정보는 2024년 6월 15일 기준으로 작성된 것이며, 법적 효력을 갖는 유권해석의 근거가 되지 않습니다. 더 자세한 정보를 원하시면 해당 링크를 참고하시기 바랍니다. (https://www.easylaw.go.kr/CSP/CnpClsMain.laf?popMenu=ov&csmSeq=508&ccfNo=3&cciNo=7&cnpClsNo=1&menuType=cnpcls&search_put=%EC%9C%A0%ED%95%99%EC%83%9D)'
content='일정 수준의 한국어 능력은 어느 정도 필요해?'
content='TOPIK 2급, 사회통합프로그램 2단계 이상 이수 또는 사전평가 41점 이상, 세종학당 중급1 이상 이수 수준이 필요합니다.'
content='나는 지금 대학교 2학년인데, 충족해야할 기준 한국어 성적이 어느정도 돼?'
content='학사 1~2학년의 경우에는 TOPIK 3급, 사회통합프로그램 3단계 이상 이수 또는 사전평가 61점 이상, 세종학당 중급1 이상 이수 수준이 필요합니다.'
content='나는 지금 대학교 3학년으로 올라가면, 충족해야할 기준 한국어 성적이 바뀌어?'
content='3학년으로 올라가면, 한국어 성적 기준은 TOPIK 4급, 사회통합프로그램 4단계 이상 이수 또